### **Итоговое задание Альмира и Соколова Александра**  
### по Проекту 6. Car Price prediction 
####  Юнит 6. Основные алгоритмы машинного обучения. Часть II (отредактирован 24.08.2020)
---
Часть 2 - Парсинг данных по списку всех марок и их моделей с сайта auto.ru 

## 1. Импорт библиотек

In [1]:
import pandas as pd
import requests, json
import time
import os
print(os.listdir('./data'))
PATH_to_file = './data/'

['all_car_brands_and_their_models_from_auto_ru.csv', 'auto_ru.csv']


### 2. Импорт данных от марках и моделях

In [2]:
df_marks_and_models = pd.read_csv(PATH_to_file+'all_car_brands_and_their_models_from_auto_ru.csv', index_col=0)
print('Кол-во марок: ', df_marks_and_models.shape[0])
display(df_marks_and_models.head(2))

Кол-во марок:  262


,Models
Mark,
AC,"['378_GT', 'ACE', 'ACECA', 'COBRA']"
ACURA,"['CL', 'CSX', 'EL', 'ILX', 'INTEGRA', 'LEGEND'..."


### 2. Отбор релевантных марок для парсинга

In [3]:
dict_of_num_cars_of_models ={'AC': 2, 'AMC': 2, 'Acura': 116, 'Alfa Romeo': 65, 
                             'Alpina': 10, 'Ariel': 1, 'Asia': 1, 'Aston Martin': 27, 
                             'Audi': 3848, 'BMW': 8295, 'BYD': 19, 'Bajaj': 1, 
                             'Bentley': 109, 'Borgward': 1, 'Brilliance': 11, 
                             'Buick': 14, 'Cadillac': 341, 'Changan': 98, 'Chery': 2574, 
                             'Chevrolet': 23883, 'Chrysler': 2313, 'Citroen': 21169, 
                             'DKW': 21, 'DS': 23, 'DW Hower': 28, 'Dacia': 29, 
                             'Daewoo': 1038, 'Daihatsu': 57, 'Daimler': 5, 
                             'Datsun': 210, 'Deco Rides': 1, 'Delage': 1, 
                             'Derways': 3, 'Dodge': 410, 'DongFeng': 22, 'Doninvest': 1, 
                             'Eagle': 2, 'Excalibur': 1, 'FAW': 70, 'Ferrari': 27, 
                             'Fiat': 235, 'Fisker': 1, 'Ford': 5265, 'Foton': 2, 
                             'GAC': 11,'GMC': 28, 'Geely': 224, 'Genesis': 64, 
                             'Great Wall': 430, 'Hafei': 6, 'Haima': 15, 'Haval': 101, 
                             'Hawtai': 4, 'Heinkel': 1, 'Honda': 1289, 'Hummer': 69, 
                             'Hyundai': 5850, 'Infiniti': 1041, 'Innocenti': 1, 
                             'Iran Khodro': 25, 'Isuzu': 39, 'JAC': 8, 'JMC': 1, 
                             'Jaguar': 314, 'Jeep': 476, 'Kia': 5885, 'LADA (ВАЗ)': 9288, 
                             'Lamborghini': 18, 'Lancia': 9, 'Land Rover': 1708, 
                             'Lexus': 1186, 'Lifan': 327, 'Lincoln': 67, 'Lotus': 1, 
                             'Luxgen': 10, 'MG': 11, 'MINI': 368, 'Mahindra': 1, 
                             'Maserati': 61, 'Maybach': 8, 'Mazda': 2508, 'McLaren': 4, 
                             'Mercedes-Benz': 8482, 'Mercury': 12, 'Metrocab': 3, 
                             'Mitsubishi': 4242, 'Mitsuoka': 3, 'Nissan': 5684, 
                             'Oldsmobile': 8, 'Opel': 3376, 'PUCH': 1, 'Packard': 2, 
                             'Peugeot': 1784, 'Plymouth': 11, 'Pontiac': 45, 
                             'Porsche': 815, 'Proton': 3, 'Ravon': 153, 'Renault': 3105, 
                             'Rolls-Royce': 47, 'Rover': 69, 'SEAT': 83, 'Saab': 158, 
                             'Saturn': 10, 'Scion': 9, 'Shanghai Maple': 2, 'Skoda': 3220, 
                             'Smart': 128, 'SsangYong': 755, 'Subaru': 1015, 
                             'Suzuki': 1103, 'Tatra': 5, 'Tesla': 45, 'Tianma': 2, 
                             'Tianye': 1, 'Toyota': 5420, 'Triumph': 1, 'Vauxhall': 1, 
                             'Volkswagen': 7144, 'Volvo': 1855, 'Vortex': 95, 
                             'Wartburg': 2, 'Willys': 2, 'ZX': 3, 'Zotye': 14, 
                             'ГАЗ': 959, 'Гоночный болид': 2, 'ЗАЗ': 253, 'ЗИЛ': 3, 
                             'ИЖ': 130, 'ЛуАЗ': 48, 'Москвич': 207, 'СМЗ': 3, 
                             'ТагАЗ': 73, 'УАЗ': 1133}

In [4]:
# уберем модели с кол-вом объявлений ниже 300 и повысим регистр индексов 
list_keys_to_delete = []
for mark in dict_of_num_cars_of_models.keys():
    if dict_of_num_cars_of_models[mark] < 300:
        list_keys_to_delete.append(mark)
    
for key in list_keys_to_delete: del dict_of_num_cars_of_models[key]

for mark in list(dict_of_num_cars_of_models.keys()):
    mark_up = mark.upper()
    dict_of_num_cars_of_models[mark_up]=dict_of_num_cars_of_models.pop(mark)

In [5]:
# посмотрим на список получившихся моделей
print(*dict_of_num_cars_of_models, sep=',')

AUDI,BMW,CADILLAC,CHERY,CHEVROLET,CHRYSLER,CITROEN,DAEWOO,DODGE,FORD,GREAT WALL,HONDA,HYUNDAI,INFINITI,JAGUAR,JEEP,KIA,LADA (ВАЗ),LAND ROVER,LEXUS,LIFAN,MINI,MAZDA,MERCEDES-BENZ,MITSUBISHI,NISSAN,OPEL,PEUGEOT,PORSCHE,RENAULT,SKODA,SSANGYONG,SUBARU,SUZUKI,TOYOTA,VOLKSWAGEN,VOLVO,ГАЗ,УАЗ


In [6]:
# Удалим отечественные модели
for key in ['LADA (ВАЗ)','ГАЗ','УАЗ']: del dict_of_num_cars_of_models[key]

In [7]:
# проверим что названия моделей совпадаю с названиями каталога сайта auto.Ru
list_marks_df = list(df_marks_and_models.index)
list_marks_to_check = []
for mark in dict_of_num_cars_of_models.keys():
    if mark not in list_marks_df:
        list_marks_to_check.append(mark)
print(f'всего релевантных марок := {len(dict_of_num_cars_of_models)}')
print('список марок, которые нужно проверить :=', *list_marks_to_check)

всего релевантных марок := 36
список марок, которые нужно проверить := GREAT WALL LAND ROVER MERCEDES-BENZ SSANGYONG


In [8]:
# исправим несоответствия
dict_of_num_cars_of_models['GREAT_WALL']=dict_of_num_cars_of_models.pop('GREAT WALL')
dict_of_num_cars_of_models['LAND_ROVER']=dict_of_num_cars_of_models.pop('LAND ROVER')
dict_of_num_cars_of_models['MERCEDES']=dict_of_num_cars_of_models.pop('MERCEDES-BENZ')
dict_of_num_cars_of_models['SSANG_YONG']=dict_of_num_cars_of_models.pop('SSANGYONG')

In [9]:
# еще раз проверим что названия моделей совпадаю с названиями каталога сайта auto.Ru
list_marks_df = list(df_marks_and_models.index)
list_marks_to_check = []
for mark in dict_of_num_cars_of_models.keys():
    if mark not in list_marks_df:
        list_marks_to_check.append(mark)
print(f'всего релевантных марок := {len(dict_of_num_cars_of_models)}')
print('список марок, которые нужно проверить :=', *list_marks_to_check)

всего релевантных марок := 36
список марок, которые нужно проверить :=


In [10]:
# запишем список релевантных марок для парсинга
list_of_relevant_marks = list(dict_of_num_cars_of_models.keys())

In [11]:
sum_of_ads = sum(dict_of_num_cars_of_models.values())
print(f"примерное кол-во строк после парсинга должно быть := {sum_of_ads}")

примерное кол-во строк после парсинга должно быть := 138518


### 3. Подготовка к парсингу

In [12]:
HEADERS = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
    'Connection': 'keep-alive',
    'Content-Length': '137',
    'content-type': 'application/json',
    'Cookie': '_csrf_token=1c0ed592ec162073ac34d79ce511f0e50d195f763abd8c24; autoru_sid=a%3Ag5e3b198b299o5jhpv6nlk0ro4daqbpf.fa3630dbc880ea80147c661111fb3270%7C1580931467355.604800.8HnYnADZ6dSuzP1gctE0Fw.cd59AHgDSjoJxSYHCHfDUoj-f2orbR5pKj6U0ddu1G4; autoruuid=g5e3b198b299o5jhpv6nlk0ro4daqbpf.fa3630dbc880ea80147c661111fb3270; suid=48a075680eac323f3f9ad5304157467a.bc50c5bde34519f174ccdba0bd791787; from_lifetime=1580933172327; from=yandex; X-Vertis-DC=myt; crookie=bp+bI7U7P7sm6q0mpUwAgWZrbzx3jePMKp8OPHqMwu9FdPseXCTs3bUqyAjp1fRRTDJ9Z5RZEdQLKToDLIpc7dWxb90=; cmtchd=MTU4MDkzMTQ3MjU0NQ==; yandexuid=1758388111580931457; bltsr=1; ',
    'Host': 'auto.ru',
    'origin': 'https://auto.ru/cars/all/',
    'Referer': 'https://auto.ru/cars/all/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0',
    'x-client-app-version': '202002.03.092255',
    'x-client-date': '1580933207763',
    'x-csrf-token': '1c0ed592ec162073ac34d79ce511f0e50d195f763abd8c24',
    'x-page-request-id': '60142cd4f0c0edf51f96fd0134c6f02a',
    'x-requested-with': 'fetch'
}

In [13]:
url = "https://auto.ru/-/ajax/desktop/listing/"

### 4. Парсинг

In [14]:
auto_ru = {}

bodyType = brand = color = fuelType = modelDate = name = numberOfDoors = productionDate = transmission = engineDisplacement = enginePower = description = mileage = complectation = lk_summary_WD = steering_wheel = owners_number = pts = custom_cleared = purchase_date = Price = []

iter_of_ads, iter_of_marks, old_iter_of_ads, sum_ads, sum_ads_from_dict = 0,0,0,0,0
space = ' '

start_time_global = time.clock()
for mark in list_of_relevant_marks:
    start_time = time.clock()
    list_of_models = json.loads(df_marks_and_models.loc[mark, 'Models'].replace("'", '"'))
    for model in list_of_models:
        for num_page in range(1, 100):
            param = {
                'catalog_filter' : [{"mark": mark, "model": model}],
                'section': "used",
                'category': "cars",
                'sort': "fresh_relevance_1-desc",
                "top_days":"90",
                "geo_radius":200,
                "geo_id":[213],
                'page': num_page
            }
            response = requests.post(url, json=param, headers = HEADERS)
            data = response.json()

            if data['offers'] != []:
                for i in data['offers']:
                    # 1. bodyType:
                    try: bodyType = i['vehicle_info']['configuration']['human_name']
                    except: bodyType = None

                    # 2. brand
                    try: brand = i['vehicle_info']['mark_info']['code']
                    except: brand = None

                    # 3. color
                    try: color = i['color_hex']
                    except: color = None

                    # 4. fuelType
                    try: fuelType = i['lk_summary'].split()[-1]
                    except: fuelType = None

                    # 5. modelDate
                    try: modelDate = i['vehicle_info']['super_gen']['year_from']
                    except: modelDate = None

                    # 6. name
                    try: name =  i['vehicle_info']['tech_param']['human_name']
                    except: name = None

                    # 7. numberOfDoors
                    try: numberOfDoors = i['vehicle_info']['configuration']['doors_count']
                    except: numberOfDoors = None

                    # 8. productionDate
                    try: productionDate = i['documents']['year']
                    except: productionDate = None

                    # 9. vehicleConfiguration
                    try: transmission = i['vehicle_info']['tech_param']['transmission']
                    except: transmission = None

                    # 10. engineDisplacement
                    try: engineDisplacement = i['vehicle_info']['tech_param']['human_name'].split()[0]
                    except: engineDisplacement = None

                    # 11. enginePower
                    try: enginePower = i['vehicle_info']['tech_param']['power']
                    except: enginePower = None

                    # 12. description
                    try: description = i['description']
                    except: description = None

                    # 13. mileage
                    try: mileage = i['state']['mileage']
                    except: mileage = None

                    # 14. Комплектация
                    try: complectation = i['vehicle_info']['complectation']
                    except: complectation = None

                    # 15. Привод
                    try: lk_summary_WD = i['lk_summary'].split(', ')[-2]
                    except: lk_summary_WD = None

                    # 16. Руль
                    try: steering_wheel = i['vehicle_info']['steering_wheel']
                    except: steering_wheel = None

                    # 17. Владельцы
                    try: owners_number = i['documents']['owners_number']
                    except: owners_number = None

                    # 18. ПТС
                    try: pts = i['documents']['pts']
                    except: pts = None

                    # 19. Таможня
                    try: custom_cleared = i['documents']['custom_cleared']
                    except: custom_cleared = None

                    # 20. Владение
                    try: purchase_date =  i['documents']['purchase_date']
                    except: purchase_date = None

                    # 21. Дата размещения объявления
                    try: start_date =  i['additional_info']['hot_info']['start_time']
                    except: start_date = None

                    # 22. Price
                    try: Price = i['price_info']['RUR']
                    except: Price = None

                    auto_ru[iter_of_ads] = {'bodyType': bodyType,
                                    'brand': brand,
                                    'color': color,
                                    'fuelType': fuelType,
                                    'modelDate': modelDate,
                                    'name': name,
                                    'numberOfDoors': numberOfDoors,
                                    'productionDate': productionDate,
                                    'vehicleConfiguration': transmission,
                                    'engineDisplacement': engineDisplacement,
                                    'enginePower': enginePower,
                                    'description': description,
                                    'mileage': mileage,
                                    'Комплектация': complectation,
                                    'Привод': lk_summary_WD,
                                    'Руль': steering_wheel,
                                    'Владельцы': owners_number,
                                    'ПТС': pts,
                                    'Таможня': custom_cleared,
                                    'Владение': purchase_date,
                                    'Дата_объявления': start_date,
                                    'Price': Price
                    }

                    iter_of_ads += 1
                    
                
            else:
                break
    iter_of_marks +=1
    temp_1 = iter_of_ads-old_iter_of_ads
    temp_2 = dict_of_num_cars_of_models[mark]
    temp_3 = temp_1/temp_2
    temp_4 = space*abs(13-len(mark)-len(str(iter_of_marks)))
    temp_5 = int(time.clock() - start_time)
    temp_8 = round(temp_3*100,2)
    temp_7 = space*abs(18-len(str(temp_1))-len(str(temp_2))-len(str(temp_8))-len(str(temp_5)))
    temp_9 = int(time.clock() - start_time_global)
    

    sum_ads += temp_1
    sum_ads_from_dict += temp_2
    temp_def = sum_ads/sum_ads_from_dict
    temp_6 = (sum_of_ads*temp_def - iter_of_ads)*temp_9/sum_ads


    print (f'{iter_of_marks}.{mark} {temp_4} ушло:{temp_5} сек., стр. {temp_1} из {temp_2} {temp_8}%,{temp_7}всего стр. {iter_of_ads}, прогноз {int(temp_6)} сек.')
    old_iter_of_ads=iter_of_ads


print (f'всего ушло: {int((time.clock() - start_time_global)/60)} мин., всего спарсили := {iter_of_ads} строк')

1.AUDI       ушло:91 сек., стр. 2215 из 3848 57.56%,  всего стр. 2215, прогноз 3223 сек.
2.BMW        ушло:158 сек., стр. 3566 из 8295 42.99%, всего стр. 5781, прогноз 2721 сек.
3.CADILLAC   ушло:34 сек., стр. 182 из 341 53.37%,    всего стр. 5963, прогноз 3004 сек.
4.CHERY      ушло:31 сек., стр. 296 из 2574 11.5%,    всего стр. 6259, прогноз 2757 сек.
5.CHEVROLET  ушло:137 сек., стр. 2236 из 23883 9.36%, всего стр. 8495, прогноз 1505 сек.
6.CHRYSLER   ушло:48 сек., стр. 168 из 2313 7.26%,    всего стр. 8663, прогноз 1570 сек.
7.CITROEN    ушло:68 сек., стр. 667 из 21169 3.15%,   всего стр. 9330, прогноз 1173 сек.
8.DAEWOO     ушло:47 сек., стр. 700 из 1038 67.44%,   всего стр. 10030, прогноз 1243 сек.
9.DODGE      ушло:28 сек., стр. 230 из 410 56.1%,     всего стр. 10260, прогноз 1289 сек.
10.FORD      ушло:182 сек., стр. 3302 из 5265 62.72%,всего стр. 13562, прогноз 1489 сек.
11.HONDA     ушло:97 сек., стр. 814 из 1289 63.15%,  всего стр. 14376, прогноз 1623 сек.
12.HYUNDAI   ушло:1

In [15]:
df_auto_ru = pd.DataFrame(auto_ru)
df_auto_ru.T.to_csv('auto_ru.csv', index=False)

In [17]:
len(df_auto_ru.T)

51599